# ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from sql_queries import *
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# We then check the number of rows in the new csv file
len(pd.read_csv('event_datafile_new.csv'))

6820

# The actual data after being aggregated would look like below 

<img src="images/image_event_datafile_new.jpg">

## Start of Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = {'class':'SimpleStrategy','replication_factor':1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Now we need to create all of the tables to run the following queries and answer the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [8]:
### Here we have drop table queries as well as create table queries within the create_table_queries and drop_table_queries

# We make sure that there's no table that have the same names as the tables we are about to use
for drop_query in drop_table_queries:
    try:
        session.execute(drop_query)
    except Exception as e:
        print(e)

# We now create the tables that are needed for our work
for create_query in create_table_queries:
    try:
        session.execute(create_query)
    except Exception as e:
        print(e)

## We will now start with the data ingestion

In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# We are now pulling the data from the aggregated file CSV below
file = 'event_datafile_new.csv'

# Now we are inserting the columns that we need so we can verify the results
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = insert_music_session_items
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)
            break

#### Do a SELECT to verify that the music session items for the first question data have been inserted into each table

In [10]:
## We'll use the select statement and print out the result
try:
    rows = session.execute("select session_id, session_item_id, artist_name, duration, song_title \
                            from music_session_items where session_id = 338 and session_item_id = 4")
except Exception as e:
    print(e)

# We will be producing the data in a clearer way by using PrettyTable Library
table = PrettyTable()
table.field_names = ["session_id", "session_item_id", "artist_name", "duration", "song_title"]
    
for row in rows:
    table.add_row([row.session_id, row.session_item_id, row.artist_name, row.duration, row.song_title])    

print(table)

+------------+-----------------+-------------+----------+---------------------------------+
| session_id | session_item_id | artist_name | duration |            song_title           |
+------------+-----------------+-------------+----------+---------------------------------+
|    338     |        4        |  Faithless  | 495.3073 | Music Matters (Mark Knight Dub) |
+------------+-----------------+-------------+----------+---------------------------------+


### We will now do the same thing for the rest of the questions answers by inserting the data into each of their respective tables

In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = insert_user_sessions
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)


In [12]:
# Here's the result for the second question

query2 = 'select user_id, session_id, session_item_id, artist_name, song_title, user_first_name, user_last_name \
          from user_sessions where session_id = 182 and user_id = 10'

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

table = PrettyTable()
table.field_names = ["user_id", "session_id", "session_item_id", "artist_name", "song_title", "user_first_name", "user_last_name"]
    
for row in rows:
    table.add_row([row.user_id, row.session_id, row.session_item_id, row.artist_name, row.song_title, row.user_first_name, row.user_last_name])
    
print(table)

+---------+------------+-----------------+-------------------+------------------------------------------------------+-----------------+----------------+
| user_id | session_id | session_item_id |    artist_name    |                      song_title                      | user_first_name | user_last_name |
+---------+------------+-----------------+-------------------+------------------------------------------------------+-----------------+----------------+
|    10   |    182     |        0        |  Down To The Bone |                  Keep On Keepin' On                  |      Sylvie     |      Cruz      |
|    10   |    182     |        1        |    Three Drives   |                     Greece 2000                      |      Sylvie     |      Cruz      |
|    10   |    182     |        2        | Sebastien Tellier |                      Kilometer                       |      Sylvie     |      Cruz      |
|    10   |    182     |        3        |   Lonnie Gordon   | Catch You Baby (Ste

In [13]:
### Query 3. Give me every user name (first and last) in my music app history who listened to 
### the song 'All Hands Against His Own'
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            session.execute(insert_song_sessions, (int(line[10]), line[9], line[1], line[4]))
        except Exception as e:
            print(e)
            break

In [14]:
# Here's the result for the 3rd question

try:
    rows = session.execute("select song_title, user_id, user_first_name, user_last_name \
                           from song_sessions where song_title = 'All Hands Against His Own'")
except Exception as e:
    print(e)

table = PrettyTable()
table.field_names = ["song_title", "user_id", "user_first_name", "user_last_name"]
    
for row in rows:
    table.add_row([row.song_title, row.user_id, row.user_first_name, row.user_last_name])  

print(table)

+---------------------------+---------+-----------------+----------------+
|         song_title        | user_id | user_first_name | user_last_name |
+---------------------------+---------+-----------------+----------------+
| All Hands Against His Own |    29   |    Jacqueline   |     Lynch      |
| All Hands Against His Own |    80   |      Tegan      |     Levine     |
| All Hands Against His Own |    95   |       Sara      |    Johnson     |
+---------------------------+---------+-----------------+----------------+


### Drop the tables before closing out the sessions

In [15]:
## Drop the table before closing out the sessions since this is a an illustration on how things are done

In [16]:
for drop_table in drop_table_queries:
    try:
        session.execute(drop_table)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()

### The full ETL process can be found within the etl.py while the table creation automation can be found within the create_tables.py